# Análisis y preprocesamiento de los datos

Se explorarán, en pequeños experimentos, distintas formas de representación de los datos del corpus WiNER (Ghaddar y Langlais 2017) para utilizarlos en la tarea de reconocimiento de entidades nombradas. Para esto se exploran distintas combinaciones de vectores de palabras como representación de una instancia de entrenamiento (Iacobacci et al. 2016).

## Descripción del Corpus WiNER

* Documents.tar.bz2 : este archivo contiene 3239540 artículos de Wikipedia repartidos en 3223 archivos. Cantidad de oraciones: 54607542. Cada archivo contiene aproximadamente 1000 artículos nombrados por sus respectivos IDs. Los artículos están indexados por su wikiID seguidos de oraciones (una por línea), donde las palabras son remplazadas por sus ids.

      ID <number>
      1234 4522 23 4 4567
      456 21 9890 123 7 0

* document.vocab : este archivo contiene el mapeo de palabras (case sensitive); el formato es: 
 
      palabra #ocurrencias
    
  El ID de cada palabra es el número de línea en la cual ocurre.

In [37]:
import pandas as pd
import numpy as np
import time
import gensim
import os
import random
import itertools
from multiprocessing import cpu_count, Pool
from gensim.models import Word2Vec, KeyedVectors

from corpus import spread_artID
from embeddings import concat_vectors, mean_vectors, fractional_decay, exponential_decay

### Cargamos los datos

In [38]:
word_mapping = pd.read_csv('./corpus_WiNER/document.vocab', sep=' ', header=None, 
                           names=['word', 'frequency'], keep_default_na=False) 
                        # con esto evito que el parser trate como nan value a algunas palabras.

/users/ekokic/thesis_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
print(word_mapping.shape)
word_mapping.head()

(2021984, 2)


,word,frequency
0,the,73411953
1,",",68044881
2,.,54241850
3,of,40550466
4,and,34602894


In [40]:
len(word_mapping['word'].unique())

2021983

### Cargamos los artículos del Documento "0"

In [41]:
doc_0 = pd.read_csv('./corpus_WiNER/Documents/0', sep='ID', engine='python', header=None, names=['sentence', 'art_ID'])

In [42]:
doc_0.shape

(130209, 2)

In [43]:
doc_0.head()

,sentence,art_ID
0,NaN,431388.0
1,650 5590 753942 12 189 243 1 2039 47 257 682 1...,NaN
2,34 23207 1523 4 57193 15 10777 14353 4 157019 ...,NaN
3,418 0 1858 101 0 1322 1 5590 18 860 729 14 92 ...,NaN
4,650 5590 1523 8 1136 15 11301 575 1156 1 2746 ...,NaN


Vamos a asociarle a cada oración el ID del artículo al cual pertenece.

Es importante recordar que el orden de las oraciones está dado por los índices del dataframe.

In [44]:
art_ID_list = doc_0['art_ID'].tolist()
art_ID = 0
for idx, elem in enumerate(art_ID_list):
    if not np.isnan(elem):
        art_ID = elem
    else:
        art_ID_list[idx] = art_ID
doc_0['art_ID'] = art_ID_list

Removemos ahora las filas con NaN que contenian los ID de los artículos inicialmente.

In [45]:
doc_0 = doc_0.dropna()

In [46]:
print('El documento contiene {} oraciones.'.format(doc_0.shape[0]))
print('El documento contiene {} artículos'.format(len(doc_0['art_ID'].unique())))

El documento contiene 128395 oraciones.
El documento contiene 1814 artículos


Hacemos que cada sentencia sea una lista de palabras codificadas y casteamos a int

In [47]:
doc_0['sentence'] = doc_0['sentence'].map(lambda x: list(map(int, x.split(' '))))

In [48]:
doc_0.head()

,sentence,art_ID
1,"[650, 5590, 753942, 12, 189, 243, 1, 2039, 47,...",431388.0
2,"[34, 23207, 1523, 4, 57193, 15, 10777, 14353, ...",431388.0
3,"[418, 0, 1858, 101, 0, 1322, 1, 5590, 18, 860,...",431388.0
4,"[650, 5590, 1523, 8, 1136, 15, 11301, 575, 115...",431388.0
5,"[152, 642, 16, 8143, 23122, 20, 0, 662955, 16,...",431388.0


Nos quedamos con un artículo

In [49]:
article = doc_0[doc_0.art_ID == 1000]
article.head()

,sentence,art_ID
2756,"[63903, 24730, 9, 7, 3035, 4104, 7584, 1, 355,...",1000.0
2757,"[3654, 16, 2502, 39413, 1, 24730, 9, 44, 3, 99...",1000.0
2758,"[24730, 35, 49, 3521, 15, 575, 1, 15, 2440, 1,...",1000.0
2759,"[152, 112, 8, 2037, 20, 52, 54, 3035, 17572, 3...",1000.0
2760,"[64, 62, 5787, 1132, 15, 0, 144, 24730, 1312, ...",1000.0


Reconstruimos la primera oración del artículo

In [50]:
def sentence_decoder(sentence, word_mapping):
    dec_sentence = []
    for idx in sentence:
        mapped_w = word_mapping.loc[idx, 'word']
        dec_sentence.append(mapped_w)
    return dec_sentence 

In [51]:
# Escribimos article.columns.get_loc('sentence') para evitar hardcodear el índice correspondiente
# a la columna 'sentence' que en este caso es 0
sentence = article.iloc[0, article.columns.get_loc('sentence')]

In [52]:
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

### Word embeddings utilizando el modelo pre-entrenado word2vec de Google

Utilizaremos la librería Gensim https://radimrehurek.com/gensim/

Cargamos Google's pre-trained Word2Vec model.

Utilizando KeyedVectors para cargar el modelo tiene la desventaja de que no se puede seguir entrenando. Pero es más eficiente que utilizar gensim.models.Word2Vec
https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors

In [53]:
start = time.time()
# model = KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)
# model.save('./models/word2vecGoogle.model')
w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
end = time.time()
print('demora: {}'.format(end-start))
# model = Word2Vec.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

demora: 9.04468035697937


In [54]:
print('Cantidad de word embeddings: {}'.format(len(w2v_model.vectors)))

Cantidad de word embeddings: 3000000


In [55]:
print('Dimensionalidad de los vectores: {}'.format(w2v_model.vector_size))

Dimensionalidad de los vectores: 300


## Exploremos distintas combinaciones de vectores de palabras

### Concatenación

Este método consiste en concatenar los vectores de palabras que rodean una palabra objetivo en un vector más grande, que tiene un tamaño igual a las dimensiones agregadas de todos las proyecciones (embeddings) individuales.

- $w_{ij}$ = peso asociado con la i-ésima dimensión del vector de la j-ésima palabra en la oración. NOTA: con los vectores de palabras de una oración se forma una matriz $w^{\space D\space x\space L}$ donde $L$ es la cantidad de palabras de esa oración.
- $D$ = dimensionalidad de los word vectors originales. Por ejemplo, al usar el modelo word2vec de Google se tiene $D$ = 300.
- $W$ = tamaño de ventana que se define como el número de palabras en un solo lado.

Nos interesa representar el contexto de la I-ésima palabra de la oración. 

La i-ésima dimensión del vector de concatenación, que tiene un tamaño de $2 W D$, se calcula de la siguiente manera:

$$ e_{i} =\begin{cases} 
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space \left\lfloor{\frac{i}{D}}\right\rfloor} & \left\lfloor{\frac{i}{D}}\right\rfloor < W \\
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space 1\space  +\space\left\lfloor{\frac{i}{D}}\right\rfloor} & c.c.
   \end{cases}$$
   

<br>
Al tomar una ventana simétrica, se realiza un relleno (padding) con ceros a izquierda y/o derecha según corresponda para mantener la misma dimesionalidad en cada nuevo vector generado.

### Promedio

Como su nombre indica, se calcula el centroide de los embeddings de todas las palabras circundantes. La fórmula divide cada dimensión en $2W$ ya que el número de palabras del contexto es dos veces el tamaño de la ventana:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} \frac{w_{ij}}{2W}$$

### Decaimiento fraccional

Una tercera estrategia para construir un vector de carácteristicas en base a los embeddings de palabras contextuales está inspirada en la forma en que Word2vec combina las palabras en el contexto. Aquí, se supone que la importancia de una palabra para nuestra representación es inversamente proporcional a su distancia respecto a la palabra objetivo.

Por lo tanto, las palabras contextuales se ponderan en función de su distancia de la palabra objetivo:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} *\frac{W - \lvert I-j\rvert}{W}$$


### Decaimiento exponencial

Funciona de manera similar al decaimiento fraccional, que le da más importancia al contexto cercano, pero en este caso la ponderación se realiza exponencialmente:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} * (1 - \alpha)^{\lvert \space I\space-\space j\space\rvert\space-\space1}$$

donde $\alpha = 1 - 0.1^{(W-1)^{-1}}$ es el parámetro de decaimiento. Elegimos el parámetro de tal manera que las palabras inmediatas que rodean a la palabra objetivo contribuyen 10 veces más que las últimas palabras en ambos lados de la ventana.

## Exploremos CoarseNE.tar.bz2

Contiene menciones anotadas automáticamente con etiquetas de entidades nombradas (PER, LOC, ORG y MISC).

El formato es:

    ID artID
    sentIdx begin end entityType
    
donde entityType[0] = PER | entityType[1] = LOC | entityType[2] = ORG | entityType[3] = MISC

In [56]:
coarseNE_0 = pd.read_csv('./corpus_WiNER/CoarseNE/0', sep='ID', engine='python', header=None, names=['named-entity', 'art_ID'])

In [57]:
print(coarseNE_0.shape)
coarseNE_0.head()

(259470, 2)


,named-entity,art_ID
0,NaN,1000.0
1,0\t0\t2\t0,NaN
2,0\t5\t6\t1,NaN
3,0\t10\t12\t0,NaN
4,1\t2\t4\t0,NaN


Aplicamos el mismo truco que utilizamos en los documentos para propagar los art_ID según corresponda

In [58]:
coarseNE_0 = spread_artID(coarseNE_0)
coarseNE_0 = coarseNE_0.dropna()

In [59]:
print('coarseNE_0 contiene {} entidades.'.format(coarseNE_0.shape[0]))
print('coarseNE_0 contiene {} artículos'.format(len(coarseNE_0['art_ID'].unique())))

coarseNE_0 contiene 257656 entidades.
coarseNE_0 contiene 1810 artículos


Creamos nuevas columnas con la info de la columna named-entity para mejor manipulación

In [60]:
def get_tag(x):
    tags = ['PER', 'LOC', 'ORG', 'MISC']
    return tags[x]

In [61]:
coarseNE_0['named-entity'] = coarseNE_0['named-entity'].map(lambda x: x.split('\t'))
coarseNE_0['senIdx'] = coarseNE_0['named-entity'].map(lambda x: int(x[0]))
coarseNE_0['begin'] = coarseNE_0['named-entity'].map(lambda x: int(x[1]))
coarseNE_0['end'] = coarseNE_0['named-entity'].map(lambda x: int(x[2]))
coarseNE_0['entityType'] = coarseNE_0['named-entity'].map(lambda x: get_tag(int(x[3])))
coarseNE_0 = coarseNE_0.drop(columns='named-entity')

In [62]:
coarseNE_0.head()

,art_ID,senIdx,begin,end,entityType
1,1000.0,0,0,2,PER
2,1000.0,0,5,6,LOC
3,1000.0,0,10,12,PER
4,1000.0,1,2,4,PER
5,1000.0,1,5,6,PER


No todos los artículos que ocurren en Documents/0 se encuentran en CoarseNE/0

Notar que esto sucede porque esos artículos no contienen entidades nombradas.

In [63]:
print('Artículos que están presentes en Documents/0 pero no en CoarseNE/0: {}'
      .format(list(set(doc_0.art_ID.unique()) - set(coarseNE_0.art_ID.unique()))))

Artículos que están presentes en Documents/0 pero no en CoarseNE/0: [431177.0, 432375.0, 432318.0, 10263.0]


In [64]:
article = doc_0[doc_0.art_ID == 431177]
for sentence in article.sentence.values:
    dec_sentence = sentence_decoder(sentence, word_mapping)
    print(' '.join(dec_sentence))

In economics , economics of location is a strategy used by firms in a monopolistic competition environment .
Unlike a product differentiation strategy , where firms make their products different in order to attract customers , the economics of location strategy causes firms to produce similar or identical products .


### Veamos como están anotadas las entidades nombradas de una oración en particular 

In [65]:
article = doc_0[doc_0.art_ID == 1000]
sentence = article.iloc[0, article.columns.get_loc('sentence')]
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

In [66]:
art_entities = coarseNE_0[coarseNE_0.art_ID == 1000]
entities_sen_0 = art_entities[art_entities.senIdx == 0]

In [67]:
for idx, row in entities_sen_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

Hercule Poirot : PER
Belgian : LOC
Agatha Christie : PER


### Ahora utilicemos los dataframes preprocesados

In [68]:
doc77_df = pd.read_pickle('./corpus_WiNER/docs_df/doc_77')
article_df = doc77_df[doc77_df.art_ID == 145492]
dec_sentence = article_df.iloc[0, article_df.columns.get_loc('sentence')]
article_df.head()

,sentence,art_ID
1,"[Belton, House, is, a, Grade, I, listed, count...",145492.0
2,"[The, mansion, is, surrounded, by, formal, gar...",145492.0
3,"[Belton, has, been, described, as, a, compilat...",145492.0
4,"[The, house, has, also, been, described, as, t...",145492.0
5,"[Only, Brympton, d'Evercy, has, been, similarl...",145492.0


In [69]:
coarseNE_77 = pd.read_pickle('./corpus_WiNER/coarseNE_df/coarseNE_77')
art_entities_df = coarseNE_77[coarseNE_77.art_ID == 145492]
sen_entities_0 = art_entities_df[art_entities_df.senIdx == 0]
art_entities_df.head()

,art_ID,senIdx,begin,end,entityType
42515,145492.0,0,0,2,LOC
42516,145492.0,0,10,11,LOC
42517,145492.0,0,12,13,LOC
42518,145492.0,0,14,15,LOC
42519,145492.0,0,16,17,LOC


In [70]:
def entityListFromSentence(senIdx, sen_length, art_entities_df):
    # We take the df with the entities of each sentence
    sen_entities_df = art_entities_df[art_entities_df.senIdx == senIdx]
    # An empty dataframe means that the sentence doesn't have any entity
    if sen_entities_df.empty:
        entities = ['O' for _ in range(sen_length)]
    else:
        entities = []
        i = 0
        for _, row in sen_entities_df.iterrows():
            while i < row['begin']:
                entities.append('O')
                i += 1
            while i < row['end']:
                entities.append(row['entityType'])
                i += 1
        while i < sen_length:
            entities.append('O')
            i += 1
    return entities

In [71]:
for idx, row in sen_entities_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

Belton House : LOC
Belton : LOC
Grantham : LOC
Lincolnshire : LOC
England : LOC


In [72]:
entity_list = entityListFromSentence(senIdx=0, sen_length=len(dec_sentence),
                                     art_entities_df=art_entities_df)
print(' '.join(article_df['sentence'][1]))
print(entity_list)

Belton House is a Grade I listed country house in Belton near Grantham , Lincolnshire , England .
['LOC', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOC', 'O', 'LOC', 'O', 'LOC', 'O', 'LOC', 'O']


In [73]:
def w2v_strategy(strategy, sentence, W, w2v_model):
    return {
        'concat': concat_vectors(sentence, W, w2v_model), 
        'mean': mean_vectors(sentence, W, w2v_model), 
        'frac_decay': fractional_decay(sentence, W, w2v_model),
        'exp_decay': exponential_decay(sentence, W, w2v_model),
    }[strategy]

In [200]:
def process_chunk(proc_chunk, f_name, art_entities_df):
    if f_name == 'w2v':
        fun = lambda sentence: w2v_strategy('exp_decay', sentence, 5, w2v_model)
        chunk_res = proc_chunk['sentence'].map(fun)
    elif f_name == 'get_entity':
        fun = lambda senIdx: entityListFromSentence(senIdx, proc_chunk.loc[senIdx, 'sen_length'],
                                                    art_entities_df)
        chunk_res = proc_chunk.index.map(fun)
    else:
        raise Exception
    chunk_res.index = proc_chunk.index
    return chunk_res

def get_proc_chunks(df, n_proc):
    chunksize = len(df) // n_proc
    proc_chunks = []
    for i_proc in range(n_proc):
        chunkstart = i_proc * chunksize
        # make sure to include the division remainder for the last process
        chunkend = (i_proc + 1) * chunksize if i_proc < n_proc - 1 else None
        proc_chunks.append(df.iloc[slice(chunkstart, chunkend)])
    return proc_chunks

def run(proc_chunks, f_name, art_entities_df):
    with Pool(processes=n_proc) as pool:
        # starts the sub-processes without blocking
        # pass the chunk to each worker process
        proc_results = [pool.apply_async(process_chunk,
                                         args=(chunk,f_name,art_entities_df))
                        for chunk in proc_chunks]
        # blocks until all results are fetched
        result_chunks = [r.get() for r in proc_results]
    return result_chunks

In [201]:
df = article_df
n_proc = 5 #cpu_count() #Number of CPU cores on your system
fun = lambda sentence: w2v_strategy('exp_decay', sentence, 5, w2v_model)
proc_chunks = get_proc_chunks(df, n_proc)
result_chunks = run(proc_chunks, 'w2v', art_entities_df)
art_vectors = pd.concat(result_chunks)
art_vectors = list(itertools.chain(*art_vectors))

In [202]:
print(len(art_vectors))

4395


In [190]:
def getVector_EntityFromArticle(article_df, art_entities_df, strategy, W, w2v_model):
    '''@return: filled DataFrame with columns {wordVector, entityType}'''
    article_df = article_df.reset_index(drop=True)
    article_df['sen_length'] = article_df['sentence'].map(lambda x: len(x))

    n_proc = 5 #cpu_count() #Number of CPU cores on your system

    proc_chunks = get_proc_chunks(article_df, n_proc)
    result_chunks = run(proc_chunks, 'w2v', art_entities_df)
    art_vectors = pd.concat(result_chunks)

    result_chunks = run(proc_chunks, 'get_entity', art_entities_df)
    art_entities = result_chunks
    # Fastest way to flatten list of arrays
    art_vectors = list(itertools.chain(*art_vectors))
    art_entities = list(itertools.chain(*art_entities))
    art_entities = list(itertools.chain(*art_entities))
    wordVector_Entity_df = pd.DataFrame({'wordVector': art_vectors, 
                                         'entityType': art_entities})

    return wordVector_Entity_df

In [191]:
start = time.time()
wordVector_Entity_df = getVector_EntityFromArticle(article_df, art_entities_df, 
                                                   strategy='exp_decay',
                                                   W=5, w2v_model=w2v_model)
end = time.time()
print('demora:', end-start)

demora: 1.6597187519073486


In [192]:
entity_list = list(wordVector_Entity_df['entityType'])
tokens = len(entity_list)
print('Cantidad de palabras: {}'.format(tokens))
n_e = len(entity_list) - entity_list.count('O')
print('Cantidad de entidades nombradas: {}'.format(n_e))
print('Porcentaje de entidades nombradas: {:.2f}%'.format(n_e/tokens*100))

Cantidad de palabras: 4395
Cantidad de entidades nombradas: 399
Porcentaje de entidades nombradas: 9.08%


En general, cada artículo contiene una proporción reducida de palabras que son entidades.

Una alternativa para evitar cómputo y uso de memoria podría ser eliminar una proporción de vectores que no representan ninguna entidad (etiquetados con 'O') a la hora de construir la matriz de palabra - entidad.

In [193]:
def drop_non_entities(df, frac):
    '''
    Remove a fraction of non entities vectors (entityType == 'O')
    df: wordVector_Entity_df
    frac: float value between 0 and 1
    @return df with a fraction of the non entities rows removed
    '''
    sample = df[df.entityType == 'O'].sample(frac=frac, random_state=77)
    df = df.drop(index=sample.index)

    return df

In [194]:
wordVector_Entity_df.head(10)

,wordVector,entityType
0,"[-0.003621502994722617, 0.012132806459215266, ...",LOC
1,"[-0.08827082592044064, -0.13598437701614993, 0...",LOC
2,"[-0.16997208299104571, 0.0033914764131283102, ...",O
3,"[-0.3915967207881669, -0.12179563739202309, 0....",O
4,"[0.048302400391625866, -0.09540661652931712, 0...",O
5,"[-0.5453598294447486, -0.15736036498200667, 0....",O
6,"[-0.12074501704763917, 0.06705739051063955, 0....",O
7,"[-0.014825387882357826, -0.25825100262032796, ...",O
8,"[-0.10727509902624327, 0.04387436488263421, 0....",O
9,"[0.07985290764610813, -0.14920335335911, -0.03...",O


In [195]:
drop_non_entities(wordVector_Entity_df.head(10), 0.50)

,wordVector,entityType
0,"[-0.003621502994722617, 0.012132806459215266, ...",LOC
1,"[-0.08827082592044064, -0.13598437701614993, 0...",LOC
5,"[-0.5453598294447486, -0.15736036498200667, 0....",O
6,"[-0.12074501704763917, 0.06705739051063955, 0....",O
8,"[-0.10727509902624327, 0.04387436488263421, 0....",O
9,"[0.07985290764610813, -0.14920335335911, -0.03...",O


In [212]:
def genWordVectors_Entity(doc_filename, coarseNE_filename, strategy, W, w2v_model):
    '''
    Creates a N x D matrix of word vectors and saves it to disk. 
    Creates an 1 x N matrix of entities and saves it to disk. 
    N is the number of words in the document.
    D is the size of each word vector.
    The entity types are: PER - LOC - ORG - MISC - O
    
    strategy: 'concat', 'mean', 'frac_decay', 'exp_decay'.
    W: window size
    w2v_model: pre-trained word2vec model
    '''
    wordVectors = []
    entityVector = []
    count = 0 # Used in the 'progress bar'    
    doc_df = pd.read_pickle('./corpus_WiNER/docs_df/'+ doc_filename)
    coarseNE_df = pd.read_pickle('./corpus_WiNER/coarseNE_df/'+ coarseNE_filename)
    art_IDs = coarseNE_df.art_ID.unique()
    
    print('starting')
    start = time.time()
    print('doc_df shape:', doc_df.shape)
    n_proc = 20 #cpu_count() #Number of CPU cores on your system
    proc_chunks = get_proc_chunks(doc_df, n_proc)
    print('len proc_chunks:', len(proc_chunks[0]))
    result_chunks = run(proc_chunks, 'w2v', coarseNE_df)
    print('finishing')
    end = time.time()
    print('demora:', end-start)
    art_vectors = pd.concat(result_chunks)
    print(type(art_vectors))
    print('shapeee:', art_vectors.shape)
    
    
    # We consider only the articles with at least one entity.
    # That's why we iterate over the coarseNE's articles.
    for art_ID in np.nditer(art_IDs):
        article_df = doc_df[doc_df.art_ID == art_ID]
        art_entities_df = coarseNE_df[coarseNE_df.art_ID == art_ID]     
        wordVector_Entity_df = getVector_EntityFromArticle(article_df, art_entities_df, 
                                                           strategy, W, w2v_model)         
        wordVector_Entity_df = drop_non_entities(wordVector_Entity_df, 0.90)
        wordVectors += list(wordVector_Entity_df['wordVector'])         
        entityVector += list(wordVector_Entity_df['entityType'])

        if count % 10 == 0: # Kind of progress bar :P
            print(count, end=' ')
        count += 1

    starting = time.time()
    np.savez_compressed('./corpus_WiNER/entity_vectors/ev_' + doc_filename + '_' + strategy,
                        entityVector)
    np.savez_compressed('./corpus_WiNER/word_vectors/wv_'+ doc_filename + '_' + strategy,
                        *wordVectors)
    finishing = time.time()
    print('tiempo de guardado:', finishing - starting)

In [213]:
start = time.time()
genWordVectors_Entity(doc_filename='doc_0', coarseNE_filename='coarseNE_0',
                      strategy='exp_decay', W=5, w2v_model=w2v_model)
end = time.time()
print('Demora total: {}'.format(end - start))

starting
doc_df shape: (128395, 2)
len proc_chunks: 6419


/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/users/ekokic/thesis_env/lib/python3.6/s

finishing
demora: 404.55298924446106
<class 'pandas.core.series.Series'>
shapeee: (128395,)
0 

KeyboardInterrupt: 

In [53]:
# word_vectors = np.load('./corpus_WiNER/word_vectors/wv_doc_0_exp_decay_.npz')
word_vectors = np.load('./corpus_WiNER/wv_doc_0_exp_decay_.npz')

In [54]:
for a_name, arr in word_vectors.iteritems():
    print(a_name, len(arr))
    break

arr_0 300


In [55]:
# entity_vector = np.load('./corpus_WiNER/entity_vectors/ev_doc_0_exp_decay_.npz')
entity_vector = np.load('./corpus_WiNER/ev_doc_0_exp_decay_.npz')

In [56]:
for a_name, arr in entity_vector.iteritems():
    print(a_name, len(arr))
    entities = list(arr)

arr_0 715816


In [57]:
entities.count('O')

296732

In [ ]:
# def createWordVectors_Entity(strategy, W):
#     '''  TODO: change description!
#     Create a N x (D+1) matrix where N is the total amount of words and 
#     D is the dimensionality of each word vector. 
#     The values of the last column are the entity type (PER - LOC - ORG - MISC - O).
    
#     strategy: 'concat', 'mean', 'frac_decay', 'exp_decay'.
#     W: window size
    
#     @return wordVectors_Entity matrix
#     '''
#     w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
#     D = w2v_model.vector_size
#     count = 0 # Used in the 'progress bar'
#     doc_filenames = os.listdir('./corpus_WiNER/docs_df/')
#     coarseNE_filenames = os.listdir('./corpus_WiNER/coarseNE_df/')
#     doc_filenames.sort()
#     coarseNE_filenames.sort()
#     #TODO: remove the following after optimization
#     starting = time.time()
    
    
#     # TODO: modularizar: que el for interno sea otro metodo
    
#     for doc_f, coarseNE_f in zip(doc_filenames, coarseNE_filenames):
#         wordVectors = []
#         entityVector = []
    
#         start = time.time()
        
#         doc_df = pd.read_pickle('./corpus_WiNER/docs_df/'+ doc_f)
#         coarseNE_df = pd.read_pickle('./corpus_WiNER/coarseNE_df/'+ coarseNE_f)
#         art_IDs = coarseNE_df.art_ID.unique()

#         end = time.time()
#         print('time 0: ', end-start)        
        
#         # We consider only the articles with at least one entity.
#         # That's why we iterate over the coarseNE's articles.
#         for art_ID in np.nditer(art_IDs):
                        
#             article_df = doc_df[doc_df.art_ID == art_ID]
#             art_entities_df = coarseNE_df[coarseNE_df.art_ID == art_ID]

#             start = time.time()            
#             wordVector_Entity_df = getVector_EntityFromArticle(article_df, art_entities_df, 
#                                                                strategy, D, W, w2v_model)
#             end = time.time()
#             print('wordVector_Entity_df: ', end-start) 
            
            
# #             start = time.time()            
#             wordVector_Entity_df = drop_non_entities(wordVector_Entity_df, 0.90)
# #             end = time.time()
# #             print('dropping: ', end-start)                     
            
#             wordVectors += list(wordVector_Entity_df['wordVector'])         
#             entityVector += list(wordVector_Entity_df['entityType'])
            
# #             if count % 10 == 0:
# #                 print(count, end=' ')
# #             count += 1

        
#         finishing = time.time()
#         print('tiempo de procesamiento:', finishing - starting)

#         starting = time.time()
#         np.savez_compressed('./corpus_WiNER/entity_vectors/ev_'+doc_f+'_'+strategy+'_', entityVector)
#         np.savez_compressed('./corpus_WiNER/word_vectors/wv_'+doc_f+'_'+strategy+'_', *wordVectors)

#         finishing = time.time()
#         print('tiempo de guardado:', finishing - starting)
#         # Probamos sin el * => guardar en un array de arrays 
# #         np.savez_compressed('./corpus_WiNER/word_vectors/wv_'+doc_f, wordVectors)
      
#         break
#     print('End process')